In [1]:
##Estimación de indicadores de agua
#Se borra todo lo que se encuentra en el entorno

rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr)

Loading required package: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

### Indicador de disponibilidad de agua diaria en la vivienda

Para este, se utilizan los datos de la Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH). Dado que esa variable se empezó a capturar en 2020, únicamente se descargan los datos de ese año y 2022.

In [3]:
url_basica<-"https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte<-"microdatos/enigh"
fin<-"_ns_viviendas_csv.zip"

In [4]:
years<-c("2020", "2022")

In [5]:
# Descargar los archivos, descomprimirlos y renombrarlos
for (i in 1:length(years)) {
  url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
  destfile <- paste0("data/", years[i], fin)

  # Descargar el archivo
  download.file(url, destfile = destfile)

  # Descomprimir el archivo
  unzip(destfile, exdir = "data")

  # Renombrar el archivo descomprimido a "viviendas_xxxx.csv"
  file.rename(
    from = "data/viviendas.csv",
    to = paste0("data/viviendas_", years[i], ".csv")
  )
}

In [6]:
  #Función de estimación
  
  estimacion <- function(path, variable) {
  # Cargar los datos
  datos <- read_csv(path)

  datos <- datos %>%
    mutate(
      across(starts_with("mat") & where(is.character),
             ~ replace_na(parse_number(.x, na = c('', 'NA', '&')), 0)),
    cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")
  
  
  
    # Crear diseño muestral
mydesign <- datos %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)
    
    
      # Estimar viviendas con agua diaria
  dispagua <- mydesign %>%
group_by(dotac_agua) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      porcentaje = survey_prop(vartype = "cv")
    ) %>%
  #Filtro de viviendas con agua diaria
    filter(dotac_agua == 1) %>%
    mutate(porcentaje = round(porcentaje * 100, 2),
           year = substr(path, 16, 19))
    
    
    
    
    if (variable == "dispagua") {
    return(dispagua)
  }
  else {
    stop("Variable no reconocida")
    }
  }

In [9]:
#Estimar indicador de disponibilidad diaria de agua
tabla_disp_agua<-map_dfr(
  list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv"),
  estimacion,
  variable = "dispagua"
)
print("Porcentaje de viviendas con disponibilidad de agua diaria")
tabla_disp_agua

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 87754 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 88823 Columns: 64
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (9): folioviv, tipo_viv, mat_techos, tipo_finan, num_dueno1, num_dueno2...
dbl (55): mat_pared, mat_pisos, antiguedad, antigua_ne, cocina, cocina_dor, ...

ℹ Use 

[1] "Porcentaje de viviendas con disponibilidad de agua diaria"


dotac_agua,viviendas,viviendas_cv,porcentaje,porcentaje_cv,year
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,2162481,0.02260787,79.88,0.01928231,2020
1,2300881,0.02524791,78.05,0.02137932,2022
